### Importing the Necessary libraries

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import warnings
warnings.filterwarnings("ignore")
import string 
from collections import Counter
!pip install nltk
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('vader_lexicon')
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.sentiment.vader import SentimentIntensityAnalyzer

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package vader_lexicon to /root/nltk_data...


###  Train -> UCI Machine Learning Repository 
###  Test -> Scraped News [THE HINDU (National)]



In [0]:
train = pd.read_csv('/content/drive/My Drive/News/OnlineNewsPopularity.csv')
test = pd.read_csv('/content/drive/My Drive/News/news_scraped.csv')

In [3]:
test.head()

,date_and_time,place,text,title
0,"April 26, 2020 13:41 IST",NaN,Families awaiting the remains of their loved o...,Coronavirus | Red tape stalls return of bodies...
1,"April 26, 2020 16:59 IST","AHMEDABAD,",In a much-awaited large-scale reshuffle in the...,Centre effects major reshuffle of officials
2,"April 26, 2020 17:10 IST","NEW DELHI ,",Ahead of Prime Minister Narendra Modi’s Monday...,Coronavirus lockdown | Set out holistic plan f...
3,"April 26, 2020 15:05 IST","Mumbai ,",Maharashtra Home Minister Anil Deshmukh on Sun...,Panel probing Wadhawans' permit will submit re...
4,"April 26, 2020 16:10 IST","New Delhi,",Prime Minister Narendra Modi will interact wit...,Coronavirus | PM Modi to interact with CMs on ...


In [4]:
train.head()

,url,timedelta,n_tokens_title,n_tokens_content,n_unique_tokens,n_non_stop_words,n_non_stop_unique_tokens,num_hrefs,num_self_hrefs,num_imgs,num_videos,average_token_length,num_keywords,data_channel_is_lifestyle,data_channel_is_entertainment,data_channel_is_bus,data_channel_is_socmed,data_channel_is_tech,data_channel_is_world,kw_min_min,kw_max_min,kw_avg_min,kw_min_max,kw_max_max,kw_avg_max,kw_min_avg,kw_max_avg,kw_avg_avg,self_reference_min_shares,self_reference_max_shares,self_reference_avg_sharess,weekday_is_monday,weekday_is_tuesday,weekday_is_wednesday,weekday_is_thursday,weekday_is_friday,weekday_is_saturday,weekday_is_sunday,is_weekend,LDA_00,LDA_01,LDA_02,LDA_03,LDA_04,global_subjectivity,global_sentiment_polarity,global_rate_positive_words,global_rate_negative_words,rate_positive_words,rate_negative_words,avg_positive_polarity,min_positive_polarity,max_positive_polarity,avg_negative_polarity,min_negative_polarity,max_negative_polarity,title_subjectivity,title_sentiment_polarity,abs_title_subjectivity,abs_title_sentiment_polarity,shares
0,http://mashable.com/2013/01/07/amazon-instant-...,731.0,12.0,219.0,0.663594,1.0,0.815385,4.0,2.0,1.0,0.0,4.680365,5.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,496.0,496.0,496.000000,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.500331,0.378279,0.040005,0.041263,0.040123,0.521617,0.092562,0.045662,0.013699,0.769231,0.230769,0.378636,0.100000,0.7,-0.350000,-0.600,-0.200000,0.500000,-0.187500,0.000000,0.187500,593
1,http://mashable.com/2013/01/07/ap-samsung-spon...,731.0,9.0,255.0,0.604743,1.0,0.791946,3.0,1.0,1.0,0.0,4.913725,4.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.799756,0.050047,0.050096,0.050101,0.050001,0.341246,0.148948,0.043137,0.015686,0.733333,0.266667,0.286915,0.033333,0.7,-0.118750,-0.125,-0.100000,0.000000,0.000000,0.500000,0.000000,711
2,http://mashable.com/2013/01/07/apple-40-billio...,731.0,9.0,211.0,0.575130,1.0,0.663866,3.0,1.0,1.0,0.0,4.393365,6.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,918.0,918.0,918.000000,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.217792,0.033334,0.033351,0.033334,0.682188,0.702222,0.323333,0.056872,0.009479,0.857143,0.142857,0.495833,0.100000,1.0,-0.466667,-0.800,-0.133333,0.000000,0.000000,0.500000,0.000000,1500
3,http://mashable.com/2013/01/07/astronaut-notre...,731.0,9.0,531.0,0.503788,1.0,0.665635,9.0,0.0,1.0,0.0,4.404896,7.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.028573,0.419300,0.494651,0.028905,0.028572,0.429850,0.100705,0.041431,0.020716,0.666667,0.333333,0.385965,0.136364,0.8,-0.369697,-0.600,-0.166667,0.000000,0.000000,0.500000,0.000000,1200
4,http://mashable.com/2013/01/07/att-u-verse-apps/,731.0,13.0,1072.0,0.415646,1.0,0.540890,19.0,19.0,20.0,0.0,4.682836,7.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,545.0,16000.0,3151.157895,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.028633,0.028794,0.028575,0.028572,0.885427,0.513502,0.281003,0.074627,0.012127,0.860215,0.139785,0.411127,0.033333,1.0,-0.220192,-0.500,-0.050000,0.454545,0.136364,0.045455,0.136364,505


In [5]:
test.dtypes

date_and_time    object
place            object
text             object
title            object
dtype: object

In [6]:
train.dtypes

url                               object
 timedelta                       float64
 n_tokens_title                  float64
 n_tokens_content                float64
 n_unique_tokens                 float64
                                  ...   
 title_subjectivity              float64
 title_sentiment_polarity        float64
 abs_title_subjectivity          float64
 abs_title_sentiment_polarity    float64
 shares                            int64
Length: 61, dtype: object

In [7]:
test.isnull().sum()

date_and_time     0
place            49
text             73
title            18
dtype: int64

Dropped rows with null values

In [0]:
test.dropna(inplace = True)

### Making the Scraped News Dataset according to the UCI Dataset (For Prediction)



In [0]:
new_test = pd.DataFrame()

In [0]:
new_test[' title'] = test.apply(lambda row: row["title"].lower().translate(str.maketrans("","",string.punctuation)), axis = 1)
new_test[' text'] = test.apply(lambda row: row["text"].lower().translate(str.maketrans("","",string.punctuation)), axis = 1)

In [0]:
new_test[' token_title'] = new_test.apply(lambda row: word_tokenize(row[' title'],"english"),axis=1)
new_test[' token_text'] = new_test.apply(lambda row: word_tokenize(row[' text'],"english"),axis=1)

In [0]:
new_test[' n_tokens_title'] = new_test.apply(lambda row: len(row[' token_title']),axis=1)
new_test[' n_tokens_content'] = new_test.apply(lambda row: len(row[' token_text']),axis=1)

In [0]:
new_test[' n_unique_tokens'] = new_test.apply(lambda row: len(Counter(word_tokenize(row[' text'],"english"))) / row[' n_tokens_content'],axis=1)

In [0]:
def remove_stopwords(tokenized_words):
  final_words = []
  for word in tokenized_words: 
	  if word not in stopwords.words('english'):
		  final_words.append(word)
  return final_words

In [0]:
new_test[' stopword_removed_content'] = new_test.apply(lambda row: 0 ,axis=1)
for i in new_test.index:
  final = remove_stopwords(new_test[' token_text'][i])
  new_test[' stopword_removed_content'][i] = final 

In [0]:
new_test[' n_non_stop_words'] = new_test.apply(lambda row: len(row[' stopword_removed_content']) / row[' n_tokens_content'] , axis = 1)

In [0]:
new_test[' n_non_stop_unique_tokens'] = new_test.apply(lambda row: len(Counter(row[' stopword_removed_content'])) / row[' n_tokens_content'] ,axis=1)

In [0]:
import datetime 
new_test[' date'] = [d.date()  for d in pd.to_datetime(test['date_and_time'])]
new_test[' weekday_is_monday'] = new_test.apply(lambda row: 0 ,axis=1)
new_test[' weekday_is_tuesday'] = new_test.apply(lambda row: 0 ,axis=1)
new_test[' weekday_is_wednesday'] = new_test.apply(lambda row: 0 ,axis=1)
new_test[' weekday_is_thursday'] = new_test.apply(lambda row: 0 ,axis=1)
new_test[' weekday_is_friday'] = new_test.apply(lambda row: 0 ,axis=1)
new_test[' weekday_is_saturday'] = new_test.apply(lambda row: 0 ,axis=1)
new_test[' weekday_is_sunday'] = new_test.apply(lambda row: 0 ,axis=1)
new_test[' is_weekend'] = new_test.apply(lambda row: 0 ,axis=1)
day_name= ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday','Sunday']
for i in new_test.index:
  day = datetime.datetime.strptime(str(new_test[' date'][i]), '%Y-%m-%d').weekday()
  #print(day_name[day]) 
  if day_name[day] == 'Monday':
    new_test[' weekday_is_monday'][i] = 1
  elif day_name[day] == 'Tuesday':
    new_test[' weekday_is_tuesday'][i] = 1
  elif day_name[day] == 'Wednesday':
    new_test[' weekday_is_wednesday'][i] = 1
  elif day_name[day] == 'Thursday':
    new_test[' weekday_is_thursday'][i] = 1
  elif day_name[day] == 'Friday':
    new_test[' weekday_is_friday'][i] = 1
  elif day_name[day] == 'Saturday':
    new_test[' weekday_is_saturday'][i] = 1
    new_test[' is_weekend'][i] = 1
  elif day_name[day] == 'Sunday':
    new_test[' weekday_is_sunday'][i] = 1
    new_test[' is_weekend'][i] = 1

In [0]:
new_test[' avg_positive_polarity'] = new_test.apply(lambda row: 0.0 ,axis=1)
new_test[' avg_negative_polarity'] = new_test.apply(lambda row: 0.0 ,axis=1)
new_test[' title_sentiment_polarity'] = new_test.apply(lambda row: 0.0 ,axis=1)

In [0]:
def sentiment_analyse(sentiment_text,index):
  score = SentimentIntensityAnalyzer().polarity_scores(sentiment_text)
  new_test[' avg_positive_polarity'][index] = score['pos']
  new_test[' avg_negative_polarity'][index] = score['neg']

In [0]:
for i in new_test.index:
  sentiment_analyse(new_test[' text'][i] , i)

In [0]:
def title_polarity(sentiment_title,index):
  score = SentimentIntensityAnalyzer().polarity_scores(sentiment_title)
  new_test[' title_sentiment_polarity'][index] = score['compound']

In [0]:
for i in new_test.index:
  title_polarity(new_test[' title'][i],i)

In [55]:
new_test.head()

,title,text,token_title,token_text,n_tokens_title,n_tokens_content,n_unique_tokens,stopword_removed_content,n_non_stop_words,n_non_stop_unique_tokens,date,weekday_is_monday,weekday_is_tuesday,weekday_is_wednesday,weekday_is_thursday,weekday_is_friday,weekday_is_saturday,weekday_is_sunday,is_weekend,avg_positive_polarity,avg_negative_polarity,title_sentiment_polarity
1,centre effects major reshuffle of officials,in a muchawaited largescale reshuffle in the c...,"[centre, effects, major, reshuffle, of, offici...","[in, a, muchawaited, largescale, reshuffle, in...",6,559,0.488372,"[muchawaited, largescale, reshuffle, central, ...",0.584973,0.406082,2020-04-26,0,0,0,0,0,0,1,1,0.063,0.003,0.0000
2,coronavirus lockdown set out holistic plan fo...,ahead of prime minister narendra modi’s monday...,"[coronavirus, lockdown, set, out, holistic, pl...","[ahead, of, prime, minister, narendra, modi, ’...",12,330,0.563636,"[ahead, prime, minister, narendra, modi, ’, mo...",0.612121,0.445455,2020-04-26,0,0,0,0,0,0,1,1,0.068,0.078,0.0000
3,panel probing wadhawans permit will submit rep...,maharashtra home minister anil deshmukh on sun...,"[panel, probing, wadhawans, permit, will, subm...","[maharashtra, home, minister, anil, deshmukh, ...",14,256,0.660156,"[maharashtra, home, minister, anil, deshmukh, ...",0.648438,0.519531,2020-04-26,0,0,0,0,0,0,1,1,0.047,0.077,0.0000
4,coronavirus pm modi to interact with cms on m...,prime minister narendra modi will interact wit...,"[coronavirus, pm, modi, to, interact, with, cm...","[prime, minister, narendra, modi, will, intera...",15,484,0.528926,"[prime, minister, narendra, modi, interact, ch...",0.607438,0.431818,2020-04-26,0,0,0,0,0,0,1,1,0.067,0.032,0.0000
6,around 300 terrorists waiting in pok for intru...,with around 300 terrorists reported to be wait...,"[around, 300, terrorists, waiting, in, pok, fo...","[with, around, 300, terrorists, reported, to, ...",12,489,0.537832,"[around, 300, terrorists, reported, waiting, a...",0.605317,0.439673,2020-04-26,0,0,0,0,0,0,1,1,0.070,0.144,-0.6249


In [25]:
train.columns

Index(['url', ' timedelta', ' n_tokens_title', ' n_tokens_content',
       ' n_unique_tokens', ' n_non_stop_words', ' n_non_stop_unique_tokens',
       ' num_hrefs', ' num_self_hrefs', ' num_imgs', ' num_videos',
       ' average_token_length', ' num_keywords', ' data_channel_is_lifestyle',
       ' data_channel_is_entertainment', ' data_channel_is_bus',
       ' data_channel_is_socmed', ' data_channel_is_tech',
       ' data_channel_is_world', ' kw_min_min', ' kw_max_min', ' kw_avg_min',
       ' kw_min_max', ' kw_max_max', ' kw_avg_max', ' kw_min_avg',
       ' kw_max_avg', ' kw_avg_avg', ' self_reference_min_shares',
       ' self_reference_max_shares', ' self_reference_avg_sharess',
       ' weekday_is_monday', ' weekday_is_tuesday', ' weekday_is_wednesday',
       ' weekday_is_thursday', ' weekday_is_friday', ' weekday_is_saturday',
       ' weekday_is_sunday', ' is_weekend', ' LDA_00', ' LDA_01', ' LDA_02',
       ' LDA_03', ' LDA_04', ' global_subjectivity',
       ' global_sent

Dropping columns which could not be generated from the Scraped News Dataset

In [0]:
x = train.drop(columns = ['url',' timedelta',' num_hrefs',' num_self_hrefs',' num_imgs',' num_videos',' average_token_length',' num_keywords',
            ' data_channel_is_lifestyle',' data_channel_is_entertainment',' data_channel_is_bus',' data_channel_is_socmed',
            ' data_channel_is_tech',' data_channel_is_world',' kw_min_min',' kw_max_min',' kw_avg_min',' kw_min_max',' kw_max_max',
            ' kw_avg_max',' kw_min_avg',' kw_max_avg',' kw_avg_avg',' self_reference_min_shares',' self_reference_max_shares',
            ' self_reference_avg_sharess',' LDA_00',' LDA_01',' LDA_02',' LDA_03',' LDA_04',' global_subjectivity',' global_sentiment_polarity',
            ' global_rate_positive_words',' global_rate_negative_words',' rate_positive_words',' rate_negative_words',' min_positive_polarity',
            ' max_positive_polarity',' min_negative_polarity',' max_negative_polarity',' title_subjectivity',' abs_title_subjectivity',
            ' abs_title_sentiment_polarity',' shares'],axis=1)
y = train[' shares']

In [28]:
x.columns

Index([' n_tokens_title', ' n_tokens_content', ' n_unique_tokens',
       ' n_non_stop_words', ' n_non_stop_unique_tokens', ' weekday_is_monday',
       ' weekday_is_tuesday', ' weekday_is_wednesday', ' weekday_is_thursday',
       ' weekday_is_friday', ' weekday_is_saturday', ' weekday_is_sunday',
       ' is_weekend', ' avg_positive_polarity', ' avg_negative_polarity',
       ' title_sentiment_polarity'],
      dtype='object')

In [29]:
new_test.columns

Index([' title', ' text', ' token_title', ' token_text', ' n_tokens_title',
       ' n_tokens_content', ' n_unique_tokens', ' stopword_removed_content',
       ' n_non_stop_words', ' n_non_stop_unique_tokens', ' date',
       ' weekday_is_monday', ' weekday_is_tuesday', ' weekday_is_wednesday',
       ' weekday_is_thursday', ' weekday_is_friday', ' weekday_is_saturday',
       ' weekday_is_sunday', ' is_weekend', ' avg_positive_polarity',
       ' avg_negative_polarity', ' title_sentiment_polarity'],
      dtype='object')

In [0]:
x_test = new_test.drop([' title',' text',' token_title',' token_text',' stopword_removed_content',' date'],axis=1)

In [31]:
x_test.head()

,n_tokens_title,n_tokens_content,n_unique_tokens,n_non_stop_words,n_non_stop_unique_tokens,weekday_is_monday,weekday_is_tuesday,weekday_is_wednesday,weekday_is_thursday,weekday_is_friday,weekday_is_saturday,weekday_is_sunday,is_weekend,avg_positive_polarity,avg_negative_polarity,title_sentiment_polarity
1,6,559,0.488372,0.584973,0.406082,0,0,0,0,0,0,1,1,0.063,0.003,0.0000
2,12,330,0.563636,0.612121,0.445455,0,0,0,0,0,0,1,1,0.068,0.078,0.0000
3,14,256,0.660156,0.648438,0.519531,0,0,0,0,0,0,1,1,0.047,0.077,0.0000
4,15,484,0.528926,0.607438,0.431818,0,0,0,0,0,0,1,1,0.067,0.032,0.0000
6,12,489,0.537832,0.605317,0.439673,0,0,0,0,0,0,1,1,0.070,0.144,-0.6249


In [32]:
x_test.columns

Index([' n_tokens_title', ' n_tokens_content', ' n_unique_tokens',
       ' n_non_stop_words', ' n_non_stop_unique_tokens', ' weekday_is_monday',
       ' weekday_is_tuesday', ' weekday_is_wednesday', ' weekday_is_thursday',
       ' weekday_is_friday', ' weekday_is_saturday', ' weekday_is_sunday',
       ' is_weekend', ' avg_positive_polarity', ' avg_negative_polarity',
       ' title_sentiment_polarity'],
      dtype='object')

In [0]:
x_test = x_test.astype(float)

### Model Used
*  RandomForest Regressor

In [0]:
from sklearn.model_selection import train_test_split
x_train,x_validate,y_train,y_validate = train_test_split(x,y,test_size=0.3)

In [37]:
from sklearn.ensemble import RandomForestRegressor
clf = RandomForestRegressor(random_state=42)
clf.fit(x_train, y_train)

RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=None, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=100, n_jobs=None, oob_score=False,
                      random_state=42, verbose=0, warm_start=False)

In [0]:
pred_validate = clf.predict(x_validate)

In [0]:
pred_validate = pd.DataFrame(clf.predict(x_validate),list(y_validate))

In [49]:
pred_validate.reset_index(level=0, inplace=True)
pred_validate_df = pred_validate.rename(index=str, columns={"index": "Actual shares", 0: "Predicted shares"})
pred_validate_df.head()

,level_0,Actual shares,Predicted shares
0,0,13200,18415.35
1,1,306100,1617.67
2,2,1300,3737.27
3,3,478,1550.58
4,4,737,1019.34


In [52]:
from sklearn.metrics import mean_squared_error
mean_squared_error(pred_validate,y_validate)

101599236.94032049

In [0]:
pred_test = clf.predict(x_test)

### Generated Predictions depicting the Virality of the *News*

In [54]:
pred_test=pd.DataFrame(clf.predict(x_test),new_test[' text'])
pred_test.reset_index(level=0, inplace=True)
pred_test = pred_test.rename(index=str, columns={"index": "News", 0: "Virality"})
pred_test

,text,Virality
0,in a muchawaited largescale reshuffle in the c...,8898.490000
1,ahead of prime minister narendra modi’s monday...,10579.426667
2,maharashtra home minister anil deshmukh on sun...,10473.890000
3,prime minister narendra modi will interact wit...,6826.530000
4,with around 300 terrorists reported to be wait...,8069.530000
...,...,...
372,prime minister narendra modi on monday told th...,10619.774332
373,the all india transporters welfare association...,11063.340000
374,the congress on monday demanded an immediate i...,10284.870000
375,a notice was served on bollywood singer kanika...,9266.785184
